## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

    @property
    def precio(self):
        return self._precio
    
    @precio.setter
    def precio(self, valor):
        if 10 <= valor <= 10000:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad
    
    @cantidad.setter
    def cantidad(self, valor):
        if 0 <= valor <= 100:
            self._cantidad = valor
    
    def valorTotal(self):
        return self.precio * self.cantidad
    
    def __repr__(self):
        return f'Producto({self.codigo}, {self.nombre}, {self.precio}, {self.tipo}, {self.cantidad})'

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada

## Analisis de los errores
- Cambiar la definicion de la clase Oferta haciendo que la descripcio sea opcional
- Llamas mal a la inicialización de la clase Oferta `Oferta.__init__(self, *arg, **kwargs)` en lugar de `super().__init__(f"Descuento {descuento}%", *args, **kwargs)`
El error está en pasar self como argumento que creando un comportamiento extraño ya que asigna la instancia que estas creando al atributo `descripcion` de la clase `Oferta`

Esto hace que la propiedad `descripcion` ya no funcione y te obligo a crear un metodo `descripcion_oferta` para poder obtener la descripcion

In [3]:
class Oferta:
    # def __init__(self, descripcion="", codigos=None, tipos=None):
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos   = tipos

    def esAplicable(self, producto, cantidad):
        codigoAplicable = producto.codigo in self.codigos
        tipoAplicable   = producto.tipo   in self.tipos
            
        return codigoAplicable or tipoAplicable
    

class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        # Oferta.__init__(self, *args, **kwargs)
        super().__init__(f"Descuento {descuento}%", *args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            totalSinDescuento = producto.precio * cantidad
            montoDescuento = totalSinDescuento * (self.descuento / 100)
            return montoDescuento
        else:
            return 0
    
    def __repr__(self):
        return f"OfertaDescuento({self.descripcion}, {self.codigos}, {self.tipos})"
    
    
class Oferta2x1(Oferta):
    def __init__(self, *args, **kwargs):
        super().__init__("Oferta 2x1", *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            unidadesGratis = cantidad // 2
            precioDescuento = unidadesGratis * producto.precio
            return precioDescuento
        else:
            return 0
        
    def __repr__(self) -> str:
        return f"Oferta2x1({self.descripcion}, {self.codigos}, {self.tipos})"


In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
class Catalogo:
    def __init__(self) :
        self.productos = []
        self.ofertas = []
    
    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, 'r') as file:
            for linea in file:
                datos = linea.strip().split(',')
                if len(datos) != 5:
                    continue
                codigo, nombre, precio, tipo, cantidad = datos
                try:
                    precio = float(precio)
                    cantidad = int(cantidad)
                except ValueError:
                    continue
                producto = Producto(codigo, nombre, precio, tipo, cantidad)
                catalogo.agregar(producto)
        return catalogo

    
    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)
    
    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None
    
    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        descuentos = []
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                descuento = oferta.calcularDescuento(producto, cantidad)
                descuentos.append(descuento)
        return max(descuentos, default=0)

    def vender(self, producto, cantidad):
        if producto in self.productos:
            if producto.cantidad >= cantidad:
                producto.cantidad -= cantidad
            else:
                raise print(f"Falta stock de {producto.nombre}")
        else:
            print("El producto no existe en el catálogo.")

    def guardar(self, archivo):
        with open(archivo, 'w') as file:
            for producto in self.productos:
                linea = f"{producto.codigo},{producto.nombre},{producto.precio},{producto.tipo},{producto.cantidad}\n"
                file.write(linea)

    def informe(self):
        tipos = set(producto.tipo for producto in self.productos)
        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades else 0

        informe = "Catálogo de productos:\n"
        informe += f"Cantidad de productos: {self.cantidadProductos}\n"
        informe += f"Cantidad de unidades: {self.cantidadUnidades}\n"
        informe += f"Precio Promedio: {precio_promedio}\n"
        informe += f"Valor total: {self.valorTotal}\n"
        informe += "Tipos de productos: " + ", ".join(tipos) + "\n"
        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += "- Oferta 2x1\n"
            elif isinstance(oferta, OfertaDescuento):
                informe += f"- Descuento: {oferta.descuento}%"
        return informe

    @property
    def valorTotal(self):
        return sum(producto.valorTotal() for producto in self.productos)
        


In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente: 
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre
        self.cuit = cuit
    
    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if valor:
            self._nombre = valor
        else:
            print("El nomre no puede estar vacio")

    
    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if self._validar_cuit(valor):
            self._cuit = valor
            

    def _validar_cuit(self, valor):
        partes = valor.split('-')
        if len(partes) != 3:
            return False

        prefijo, numero, digito_verificador = partes

        if len(prefijo) != 2 or not prefijo.isdigit():
            return False

        if len(numero) != 8 or not numero.isdigit():
            return False

        if len(digito_verificador) != 1 or not digito_verificador.isdigit():
            return False

        return True

In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

El nomre no puede estar vacio
Prueba pasada exitosamente!


In [9]:
class Factura:
    _ultimo_numero = 0

    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = self.nuevoNumero()
        self.items = {}

    def agregar(self, producto, cantidad):
        if producto in self.catalogo.productos:
            if producto in self.items:
                self.items[producto] += cantidad
            else:
                self.items[producto] = cantidad
            self.catalogo.vender(producto, cantidad)
        else:
            print(f"Error: El producto {producto.nombre} no está en el catálogo.")

    @property
    def cantidadProductos(self):
        return len(self.items)

    @property
    def cantidadUnidades(self):
        return sum(self.items.values())

    @property
    def subtotal(self):
        total = 0
        for producto, cantidad in self.items.items():
            total += producto.precio * cantidad
        return total

    @property
    def descuentos(self):
        total_descuentos = 0
        for producto, cantidad in self.items.items():
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            total_descuentos += descuento
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        lineas = []
        lineas.append(f"Factura N°: {self.numero}")
        lineas.append(f"Cliente: {self.cliente.nombre}")
        lineas.append(f"CUIT: {self.cliente.cuit}")
        lineas.append("\nDetalle de productos:")
        for producto, cantidad in self.items.items():
            total_producto = producto.precio * cantidad
            lineas.append(f"{cantidad} {producto.nombre} @ {producto.precio} = {total_producto}")
            descuentos_aplicables = []
            for oferta in self.catalogo.ofertas:
                if oferta.esAplicable(producto, cantidad):
                    descuentos_aplicables.append(oferta.descripcion)
            if descuentos_aplicables:
                lineas.append("Descuentos aplicados: " + ", ".join(descuentos_aplicables))
        lineas.append(f"\nSUBTOTAL: {self.subtotal:.2f}")
        lineas.append(f"DESCUENTOS: {self.descuentos:.2f}")
        lineas.append(f"TOTAL: {self.total:.2f}")
        return "\n".join(lineas)


In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
